In [1]:
import pycaret
import numpy as np
import pandas as pd
import warnings

from pycaret.classification import *
from pycaret.classification import load_model 
from imblearn.over_sampling import *

pd.set_option("display.max_columns", None)
warnings.filterwarnings('ignore')


In [2]:
df = pd.read_csv('data/q22023.csv')
df = df.head(100)

In [3]:
df = df.dropna(subset=['Act_date'])

#Change date column type to datetime
df['Act_date'] = pd.to_datetime(df['Act_date'])

df.insert(1, 'Main Line', " ")
df.loc[df["Prod"] == "Hala Pay As You Talk", "Main Line"] = "Ooredoo Prepaid"
df.loc[df["Prod"] == "Shahry Voice", "Main Line"] = "Ooredoo Postpaid"
df.columns = df.columns.str.replace(r'AVG_of_', '')

In [4]:
saved_lr = load_model('saved_model') 

def predict_large_dataframe(df, model, chunk_size=10000, verbose=True):

    #split the dataframe into equal parts
    num_chunks = np.ceil(len(df)/chunk_size)
    df_parts = np.array_split(df, num_chunks)

    if verbose:
        print(f"Total chunks: {int(num_chunks)}")

    #predict each part using the model
    predicted_parts = []
    for idx, part in enumerate(df_parts):
        predicted_part = predict_model(model, data = part)
        predicted_parts.append(predicted_part)

        if verbose:
            print(f"Predicted chunk {idx + 1} of {int (num_chunks)}")

    # Concatenate the predicted parts into a single dataframe
    predicted_df = pd.concat(predicted_parts, ignore_index = True)
    return predicted_df

Transformation Pipeline and Model Successfully Loaded


In [30]:
predicted_data = predict_large_dataframe(df, saved_lr, verbose=True)

predicted_data.loc[predicted_data["prediction_label"] == 1, "prediction_label"] = "Plug & Play-ers"
predicted_data.loc[predicted_data["prediction_label"] == 2, "prediction_label"] = "Price Conscious Net-Poolers"
predicted_data.loc[predicted_data["prediction_label"] == 3, "prediction_label"] = "Pragmatic Prime-timers"
predicted_data.loc[predicted_data["prediction_label"] == 4, "prediction_label"] = "Connected Socialites"
predicted_data.loc[predicted_data["prediction_label"] == 5, "prediction_label"] = "Traditional Qataris"

predicted_data['prediction_label'].value_counts(normalize=True).sort_index()

Total chunks: 1
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=0.6, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
Predicted chunk 1 of 1


Connected Socialites           0.12
Plug & Play-ers                0.30
Pragmatic Prime-timers         0.18
Price Conscious Net-Poolers    0.31
Traditional Qataris            0.09
Name: prediction_label, dtype: float64

In [14]:
predicted_data.to_csv('q22023_tagged.csv')